####PREREQUISITES### 
Behold, no experience with terminal means you will not be able to follow this tutorial. Even if setting up simulations is not your main job, you clearly felt the need to setup a simulation or two. I strongly recommend getting familiar with the basic terminal commands before proceeding further. The tasks are much easier to complete!!! Trust me I have done that transition.

Also I assume you know the meaning of a simulation. If not, I do not recemmonned starting with membrane proteins. Check the web explaining the basics of a simulation first. 

Have VMD and GROMACS installed. If you are using bash set this variable:
export VMD=/path/to/your/vmd_executable


# MEMBRANE PROTEIN SIMULATION: GLIC
Setting up membrane protein simulations is trickier than preparing a soluble protein. However the basic needs are the same. Meaning that you need coordinate files (protein and lipid) and their corresponding topologies. Topology files for lipids is provided. If you have no prior experience with simulations I suggest not meddling with the gromacs default topology folder. For this reason we will need two set of forcefield files: Amber99sb-ILDN and amber99sb-ILDN_berger. I have provided these folders to you, along with other files.

Before we start lets create a project directory named GLIC_pH70. The protein we are going to use, as the name of the folder states, is called GLIC. GLIC is a pH-gated cationic channel, homologous to cys-loop receptors. pH-gated here means the activation is done by increasing the pH from 7.0 to 4.0. There for we named the folder pH70. After you have gone through this tutorial and enjoyed it, you can try changing the protonation states of some residues and setup another simulation as a practice. Back to business...

Now within GLIC_pH70 create the following folders: *prep, run-files, em, equi, prod*. *prep* folder is our working directory to setup the simulation box. *run-files* is the directy we will place the final files. In theory this is just a duplication of the files within *prep* but things gets messy there. So it is a good practice to have a clean copy of the basic files in a seperate folder. *em, equi,* and *prod* folders will contain the minimisation, equilibration and the production runs. 
  
## PREPARE THE COORDINATES AND THE TOPOLOGY 

Go a head and download the tutorial file and extract. Within the extracted folder you will find two forcefield folders. Copy them over to the prep folder. Both of them are needed. One is the original amber99sb-ILDN folder. The other one is extended with the lipid parameters.

I have provided two coordinate files ending with .pdb. 4NPQ_BA1.pdb is the first biological assembly of the GLIC 4NPQ structure. Crystal structures often contain other molecules than protein or missing residues/atoms. I have cleaned the crystal structure from extra molecules and other biological assemblies. Then used modeller to fix the missing backbone&sidechain atoms. There are other ways to fix the structure. But this will not be covered in this tutorial. The second coordinate file contains Berger POPC lipids. It is a bilayer consists of 332 lipids. This lipid patch is just enough size to accomodate GLIC. Copy both files to prep folder. 

You will also find a .tcl script and gmembed.mdp, gmembed.dat files. Copy these to the prep folder too. I will explain their job below. 

Now the real fun begins; load these two structures to your favorite visualization software. You see that these two structures are not aligned. In order to place the protein into the membrane first we need to align them. Our membrane patch expands on the xy plane. That means we need to align the transmembrane domain to its 3rd principal axis. 

If you are adventurous you can try to do this manually :-) If you want to a short cut here are some awesome tools from gromacs and a beatiful VMD script.

-First we align the protein on its principal axes and center it at 0 0 0
-Next we rotate it by 90&deg; around the y axis. These steps aligns the protein channel with the z axis. Enables us to place the transmembrane domain to the bilayer correctly.
-As the last step we move/translate the protein into the bilayer. The .tcl script moves the protein by aligning the the center of the trasmembrane domain to the center of the bilayer.

In [ ]:
gmx editconf -f 4NPQ_BA1.pdb -o GLIC_princ.pdb -center 0 0 0 -princ <<EOF
1
EOF

gmx editconf -f GLIC_princ.pdb -o GLIC_princ_rotate.pdb -rotate 0 90 0

$VMD -dispdev text -e protein_center_to_lipids.tcl -args GLIC_princ_rotate.pdb popc_332.pdb GLIC_princ_rotate_translate

If you load the output generated with above commands to a visualisation software, you will be able to understand better what each steps does. 

Now we have our protein placed at the same plane as our bilayer. However they are still two seperate files. Before we concatanate them we need to process the protein file further and obtain a topology file for it. For lipids we already have the topology files within the forcefield directory. pdb2gmx is the tool in GMX that creates the protein topologies for us. Since we want to run a GLIC simulation at pH 7.0, no need to assign different protonation states. 
pdb2gmx will ask for (1) a forcefield choice and (2) a water model. When prompted choose 1 (corresponds to amber99s-ildn_berger forcefield) and 1 (TIP3P water model).

In [ ]:
gmx pdb2gmx -f GLIC_princ_rotate_translate.pdb -o GLIC_pH70.pdb -ignh -vsite hydrogens -ter

Note that in the above command we have ignored the hydrogens within our input structure (ignh) and added new hydrogens along with virtual sites. Virtual sites are required to speed up the calculations. You can read more about it on GMX website.

Now we have a protein structure that can be combined with the lipid file.

In [ ]:
head -4 popc_332.pdb > GLIC_pH70_popc.pdb
grep 'ATOM' GLIC_pH70.pdb >> GLIC_pH70_popc.pdb
grep 'POPC' popc_332.pdb >> GLIC_pH70_popc.pdb
echo 'END' >> GLIC_pH70_popc.pdb

Next we create a box and solvate it. The bilayer file I have provided already contains the box dimensions required. So all we need to do is to center the system within that box and add waters. Adding waters to bilayer systems are a bit tricky. But there are several ways to make it happen. I will use the method where we increase the vdW radius of the POPC carbons.

In [ ]:
gmx editconf -f GLIC_pH70_popc.pdb -o GLIC_pH70_popc_center.pdb -c -bt cubic
cp /usr/local/gromacs/share/gromacs/top/vdwradii.dat b_vdwradii.dat
sed '/; Water charge sites/ a\
POP  C     0.45 \
' b_vdwradii.dat > vdwradii.dat
gmx solvate -cp GLIC_pH70_popc_center.pdb -cs -o GLIC_pH70_popc_center_sol.pdb
rm vdwradii.dat

Now take a look at the end file GLIC_pH70_popc_center_sol.pdb and make sure that the bilayer is not invaved by ton of water molecules. Altough above method worked for me everytime, it is better to make sure then cry over a setup not working.

At this point we still have overlapping lipids with the protein. Also 4NPQ structure of GLIC is not really an ideal structure. So if you choose to look at it with cartoon representation you will not see parts of the protein. Do not be alarmed. The protein is there! Change the representation to lines/licorice or speheres then you will see it.

If everything looks ok proceed to run g_membed to remove overlapping molecules. Since g_membed uses mdrun, we need to update the topology file to include lipids and water. After g_membed run we add more waters to the system so that the pore area previously occupied by lipids also gets hydrated. 

In [ ]:
SOL_NR=`grep "OW" GLIC_pH70_popc_center_sol.pdb | wc | awk '{printf $1}'`
LIPID_NR=`grep "POPC" GLIC_pH70_popc_center_sol.pdb | wc | awk '{printf $1/52}'`

grep -v "Protein_chain_[B-E]" topol.top > GLIC_pH70_popc_sol.top
sed -i '' '/topol_Protein_chain_A.itp"/ a\
#include "amber99sb-ildn-berger.ff/popc.itp" \
' GLIC_pH70_popc_sol.top
sed -i '' 's/Protein_chain_A     1/Protein_chain_A     5/g' GLIC_pH70_popc_sol.top
echo 'POPC            '$LIPID_NR >> GLIC_pH70_popc_sol.top
echo 'SOL             '$SOL_NR >> GLIC_pH70_popc_sol.top
sed -i '' 's/amber99sb-ildn.ff/amber99sb-ildn-berger.ff/g' GLIC_pH70_popc_sol.top

mkdir -p g_membed
cp gmembed.mdp gmembed.dat g_membed/
cd g_membed
rm *.tpr *.pdb
touch box.mdp 
gmx grompp -f box.mdp -c ../GLIC_pH70_popc_center_sol.pdb -p ../GLIC_pH70_popc_sol.top -o b4membed.tpr
gmx trjconv -f ../GLIC_pH70_popc_center_sol.pdb -o b4membed.pdb -s b4membed -ur rect -pbc mol<<EOF
0
EOF

gmx make_ndx -f ../GLIC_pH70_popc_center_sol.pdb -o index_before_gmembed.ndx<<EOF
name 13 POPC
q
EOF

gmx grompp -f gmembed.mdp -c b4membed.pdb -o gmembed.tpr -maxwarn 1 -n index_before_gmembed -p ../GLIC_pH70_popc_sol.top -v

gmx mdrun -nt 1 -s gmembed.tpr -membed gmembed.dat -c GLIC_pH70_popc_further_solvate.pdb -mn index_before_gmembed -v <<EOF
1
13
EOF

cp /usr/local/gromacs/share/gromacs/top/vdwradii.dat b_vdwradii.dat
sed '/; Water charge sites/ a\
POP  C     0.45 \
' b_vdwradii.dat > vdwradii.dat

gmx solvate -cp GLIC_pH70_popc_further_solvate.pdb -cs -o GLIC_pH70_POPC_SOL.pdb
rm vdwradii.dat

Next step is to update the topology with the new number of lipids and water molecules. After that we will place ions to our system using ionic strength of 0.1M . When adding ions one can use the box volume, however membrane proteins such as GLIC contain a huge extracellular domain that cuts your solvent area by half, at least. So instead I am using molality to determine the number of ions. We also need to now the charge of the protein so that we can neutralize it. You would see the charge of the protein when you run pdb2gmx. But if you haven't been paying attention to the terminal output at that step you have no idea. When setting up a new system it is a good practice to examine the printed info from pdb2gmx, not just for system charge but for disulfide bridge formations, protonation states on histidines, number of chains/residues, etc. But don't worry we won't go back. We can find the total system charge with the commands below, and adjust the number of ions accordingly. 

In [ ]:
SOL_NR2=`grep "OW" GLIC_pH70_POPC_SOL.pdb | wc | awk '{printf $1}'`
LIPID_NR2=`grep "POPC" GLIC_pH70_POPC_SOL.pdb | wc | awk '{printf $1/52}'`

cp ../GLIC_pH70_popc_sol.top GLIC_pH70_POPC_SOL.top
sed -i '' "s/$SOL_NR/$SOL_NR2/g" GLIC_pH70_POPC_SOL.top
sed -i '' "s/$LIPID_NR/$LIPID_NR2/g" GLIC_pH70_POPC_SOL.top

SYS_CHARGE=`grep qtot ../topol_Protein_chain_A.itp | tail -1 | awk '{print $11 * 5}'`

echo $SYS_CHARGE
ION_NUMBER=`echo "scale=9; (0.1 / 55.5)*$SOL_NR2" | bc `
ION_NUMBER=${ION_NUMBER%%.*}

if [ $SYS_CHARGE -gt 0 ]; then
        let NEG_ION=$ION_NUMBER+$SYS_CHARGE
        let POS_ION=$ION_NUMBER
        echo "CL- :"$NEG_ION" Na+ :"$POS_ION
elif [ $SYS_CHARGE -lt 0 ]; then
        let NEG_ION=$ION_NUMBER
        let POS_ION=$ION_NUMBER-$SYS_CHARGE
        echo "CL- :"$NEG_ION" Na+ :"$POS_ION
else
        let NEG_ION=$ION_NUMBER
        let POS_ION=$ION_NUMBER
        echo "CL- :"$NEG_ION" Na+ :"$POS_ION
fi

cd ../
cp g_membed/GLIC_pH70_POPC_SOL.top GLIC_pH70_POPC_SOL_ION.top
touch ion.mdp
gmx grompp -f ion.mdp -c g_membed/GLIC_pH70_POPC_SOL.pdb -p GLIC_pH70_POPC_SOL_ION.top -o 4ion.tpr
gmx genion -s 4ion.tpr -nn $NEG_ION -np $POS_ION -p GLIC_pH70_POPC_SOL_ION.top -o GLIC_pH70_POPC_SOL_ION.pdb<<EOF
15
EOF


Now we have a coordinate system and a topology file ready. And we are almost done with our preperations. Membrane protein systems might require a bit longer equilibration as we have a less homogenous system. To prevent protein changing its conformation abruptly during that equilibration we will place position restraints on all the atoms execpt hydrogens. These restraints will then be gradually released. For GMX to apply these restraints we have to create those files. Run the commands below to do just that. 

In [ ]:
echo '; Include Position restraint file
#ifdef POSRES_HEAVY
#include "posre_Protein_chain_A.itp"
#endif
                                                                                                                                                                                  
; Include Position restraint file
#ifdef POSRES_BACKBONE
#include "posre_Protein_chain_A_backbone.itp"
#endif
                                                                                                                                                                                  
; Include Position restraint file                                                                                                                                                 
#ifdef POSRES_CA
#include "posre_Protein_chain_A_ca.itp"
#endif' >> topol_Protein_chain_A.itp

gmx make_ndx -f GLIC_pH70_POPC_SOL_ION.pdb -o posre.ndx<<EOF
keep 4
0 & chain A 
0 & chain A & a CA
q
EOF

gmx genrestr -f GLIC_pH70_POPC_SOL_ION.pdb -n posre.ndx -o posre_Protein_chain_A_backbone<<EOF
1
EOF

gmx genrestr -f GLIC_pH70_POPC_SOL_ION.pdb -n posre.ndx -o posre_Protein_chain_A_ca<<EOF
2
EOF

cp *chain_A*itp ../run-files/
cp GLIC_pH70_POPC_SOL_ION.??? ../run-files/
cp -r amber99sb*.ff ../run-files/

And we are done! 